[API documentation](https://dev.elsevier.com/documentation/ScopusSearchAPI.wadl)


In [34]:
import requests
import pandas as pd
import sddk

In [35]:
# to communicate with google spreadsheet...
import gspread
from gspread_dataframe import get_as_dataframe
from gspread_dataframe import set_with_dataframe
from google.oauth2 import service_account # based on google-auth library
import sddk

s = sddk.cloudSession("sciencedata.dk")
# establish connection with gogglesheets...
file_data = s.read_file("https://sciencedata.dk/files/ServiceAccountsKey.json", "dict") # or load it from a local storage: json.load(open("../../ServiceAccountsKey.json", "r"))
credentials = service_account.Credentials.from_service_account_info(file_data)
gc = gspread.Client(auth=credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']))
mops_data = gc.open_by_url("https://docs.google.com/spreadsheets/d/1VbCIAJssHKV9hlRTwzVFfm40CGnHesq53KXjv2qy4OM/edit?usp=sharing")

endpoint variable has been configured to: https://sciencedata.dk/files/


In [46]:
apikey = input("go to https://dev.elsevier.com/documentation/ScopusSearchAPI.wadl, generate your API key, copy it and insert it here: ")

In [6]:
api_address = "https://api.elsevier.com/content/search/scopus?"
auth_params = "apiKey=" + apikey  # {0}&access_token={1}".format(apikey, access_token)
base_url = api_address + auth_params + "&query="
print(base_url)

https://api.elsevier.com/content/search/scopus?apiKey=4404b4d72ab1855cbe109fc606745d66&query=


In [7]:
query = "religion"
base_url + query

'https://api.elsevier.com/content/search/scopus?apiKey=4404b4d72ab1855cbe109fc606745d66&query=religion'

In [14]:
requests.get(base_url + query).json()["search-results"]["opensearch:totalResults"]

'635748'

In [16]:
query= "TITLE-ABS-KEY ( \"Paul the Apostle\"  OR  \"Apostle Paul\"  OR  \"Paul of Tarsus\"  OR  \"Pauline liter\" )"
resp = requests.get(base_url + query)
resp.json()["search-results"]["opensearch:totalResults"]

'547'

In [18]:
query= 'TITLE-ABS-KEY ( "Paul the Apostle"  OR  "Apostle Paul"  OR  "Paul of Tarsus"  OR  "Pauline literature"  OR  "Paul\'s epistles"  OR  "Paul\'s letters"  OR  "letters of Paul" )'
resp = requests.get(base_url + query)
resp.json()["search-results"]["opensearch:totalResults"]

'1075'

In [32]:
phrases_data = []
phrases = ["paul the apostle",
           "apostle paul",
           "paul of tarsus",
           "pauline literature",
           "paul\'s epistle",
           "Paul\'s letter",
           "letter of Paul",
           "epistle of paul"]

for phrase in phrases:
    cats_data = {}
    cats_data["phrase"] = phrase
    for cat in ["TITLE", "ABS", "KEY"]:
        query = cat + " ( \"{0}\")".format(phrase)
        print(query)
        resp = requests.get(base_url + query)
        cats_data[cat] = resp.json()["search-results"]["opensearch:totalResults"]
    phrases_data.append(cats_data)

TITLE ( "paul the apostle")
ABS ( "paul the apostle")
KEY ( "paul the apostle")
TITLE ( "apostle paul")
ABS ( "apostle paul")
KEY ( "apostle paul")
TITLE ( "paul of tarsus")
ABS ( "paul of tarsus")
KEY ( "paul of tarsus")
TITLE ( "pauline literature")
ABS ( "pauline literature")
KEY ( "pauline literature")
TITLE ( "paul's epistle")
ABS ( "paul's epistle")
KEY ( "paul's epistle")
TITLE ( "Paul's letter")
ABS ( "Paul's letter")
KEY ( "Paul's letter")
TITLE ( "letter of Paul")
ABS ( "letter of Paul")
KEY ( "letter of Paul")
TITLE ( "epistle of paul")
ABS ( "epistle of paul")
KEY ( "epistle of paul")


In [33]:
scopus_phrases_overview = pd.DataFrame(phrases_data)
scopus_phrases_overview

,phrase,TITLE,ABS,KEY
0,paul the apostle,16,45,25
1,apostle paul,84,308,67
2,paul of tarsus,15,37,19
3,pauline literature,3,13,12
4,paul's epistle,28,67,4
5,Paul's letter,102,280,25
6,letter of Paul,37,73,7
7,epistle of paul,0,26,3


In [36]:
set_with_dataframe(mops_data.add_worksheet("scopus_phrases_overview", 1,1), scopus_phrases_overview)

In [43]:
phrases_combined = "(\"" + "\") OR (\"".join(phrases) + "\")"
print(phrases_combined)

("paul the apostle") OR ("apostle paul") OR ("paul of tarsus") OR ("pauline literature") OR ("paul's epistle") OR ("Paul's letter") OR ("letter of Paul") OR ("epistle of paul")


In [47]:
query = "TITLE-ABS-KEY (" + phrases_combined + " )"
resp = requests.get(base_url + query)
resp.json()

{'search-results': {'opensearch:totalResults': '1102',
  'opensearch:startIndex': '0',
  'opensearch:itemsPerPage': '25',
  'opensearch:Query': {'@role': 'request',
   '@searchTerms': 'TITLE-ABS-KEY (("paul the apostle") OR ("apostle paul") OR ("paul of tarsus") OR ("pauline literature") OR ("paul\'s epistle") OR ("Paul\'s letter") OR ("letter of Paul") OR ("epistle of paul") )',
   '@startPage': '0'},
  'link': [{'@_fa': 'true',
    '@ref': 'self',
    '@href': 'https://api.elsevier.com/content/search/scopus?start=0&count=25&query=TITLE-ABS-KEY+%28%28%22paul+the+apostle%22%29+OR+%28%22apostle+paul%22%29+OR+%28%22paul+of+tarsus%22%29+OR+%28%22pauline+literature%22%29+OR+%28%22paul%27s+epistle%22%29+OR+%28%22Paul%27s+letter%22%29+OR+%28%22letter+of+Paul%22%29+OR+%28%22epistle+of+paul%22%29+%29',
    '@type': 'application/json'},
   {'@_fa': 'true',
    '@ref': 'first',
    '@href': 'https://api.elsevier.com/content/search/scopus?start=0&count=25&query=TITLE-ABS-KEY+%28%28%22paul+the+apo